<a href="https://colab.research.google.com/github/tamirmal/advanced_topics_in_learning/blob/master/ex3q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
GDRIVE_PATH='/gdrive/MyDrive/Colab_TAU/ComputationalLearning/'

!pip3 -qq install ipdb

Mounted at /gdrive
     |████████████████████████████████| 788kB 6.8MB/s 
     |████████████████████████████████| 368kB 10.6MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.22.0 which is incompatible.


In [2]:
import numpy as np
Xtrain = np.loadtxt(GDRIVE_PATH+'MNIST_train_images.csv', delimiter=',')
Ytrain = np.loadtxt(GDRIVE_PATH+'MNIST_train_labels.csv', delimiter=',')
Xtest = np.loadtxt(GDRIVE_PATH+'MNIST_test_images.csv', delimiter=',')
Ytest = np.loadtxt(GDRIVE_PATH+'MNIST_test_labels.csv', delimiter=',')

print(Xtrain.shape)
print(Ytest)
print(Ytest.shape)

(11696, 784)
[-1. -1. -1. ... -1.  1. -1.]
(1954,)


In [20]:
##
## this is a vectorization attemp
## the idea is to try all possible h in one go
## for a fixed theta
def vec_err_by_distribution(theta, p, S, dir):
  Sx, Sy = S
  assert len(Sx) == len(Sy)
  assert len(p) == len(Sx)
  assert len(Sx[0] == 784)
  assert dir in [-1, 1]

  # over entire sample
  # at each possible pixel idx, check if its smaller than theta (1 or 0)
  # example for a given example:
  #  x = (0, 0, ... 100, 105, 105, .... 0, 0) // len(x)=784
  #  h_of_x = (0 < theta, 0 < theta, ..., 100 < theta, 105 < theta, 105 < theta, ... , 0 < theta, 0 < theta)
  #  h_of_x = (T, T, ... , F, F, F, ... , T, T)
  # for all examples we wil get a matrix of T / F at each idx
  # then apply 'weigth' of each example - by distribution p
  bool_h_of_x = (Sx <= theta)
  # given back in T or F - we ned to convert to -1 / 1
  h_of_x = np.ones_like(bool_h_of_x) * -dir # init template array to '-1 * dir'
  h_of_x[bool_h_of_x] = dir # where its true - set to dir
  #print(h_of_x.shape)

  # compare to Sy - need to convert vec of labels to matrix of labels
  Sy_tile = np.tile(Sy, (len(Sx[0]),1))
  Sy_tile = np.transpose(Sy_tile)
  err = (Sy_tile != h_of_x)
  err = err.astype(np.uint8)
  #print(err.shape)

  # our target is to find h that minimizes err by some distribution
  # so first apply the distribution for all samples
  p_tile = np.tile(p, (len(Sx[0]),1))
  p_tile = np.transpose(p_tile)
  #print(f"p={p.shape}, p_tile={p_tile.shape}")
  weighted_err = err * p_tile

  ## now after applying dist - we need to calc the error by sum
  sum_weighted_err = np.sum(weighted_err, axis=0)
  assert len(sum_weighted_err) == len(Sx[0])
  #print(f"sum_w_err={sum_weighted_err.shape}")
  
  ## Look which idx gave lowest err
  # find which pixel idx has 'best' err
  best_h = np.argmin(sum_weighted_err)
  #print(f"best_h is at idx={best_h}, err={sum_weighted_err[best_h]}")
  h_of_x = h_of_x[:, best_h]
  #print(f"h_of_x={h_of_x}")
  return best_h, sum_weighted_err[best_h], h_of_x

def Adaboost(S, T):
  Sx, Sy = S

  m = len(Sx)
  p = np.ones(m) / m
  H_T = []
  for t in range(T):
    EPSILON = 1E-5
    if np.sum(p) > (1 + EPSILON) or np.sum(p) < (1 - EPSILON):
      print(f"sum(p)={np.sum(p)}")
      assert 0

    #Find ht = arg minh∈H Px∼pt[h(x) != y].
    ## lets analyze this ...
    ## h(x) != y   ===> this is the error / loss
    ## x ~ pt      ===> by some sample distribution
    ## so for some h(theta,j), we need to calculate the err for each sample and the "weigth" it by the distribution vector p
    ##
    ## note - these nested loops over dir/theta can be paralleld with 'workers' in scikit-learn n_jobs (uses more cpu's)
    best_h = {'idx': None, 'theta': None, 'dir': None, 'err': 1, 'h_of_x': None}
    for dir in [-1, 1]:
      for theta in range(256):
        best_h_idx, err, h_of_x = vec_err_by_distribution(theta, p, S, dir)
        if err < best_h['err']:
          best_h['theta'] = theta
          best_h['idx'] = best_h_idx
          best_h['err'] = err
          best_h['h_of_x'] = h_of_x
          best_h['dir'] = dir
    # end

    e_t = best_h['err']
    a_t = 0.5*np.log((1-e_t)/e_t)
    best_h['a_t'] = a_t
    H_T.append(best_h)
    #update p
    h_of_x = best_h['h_of_x']
    h_of_x_aux = np.exp(h_of_x * Sy * -1.0 * a_t)
    p_MUL_h_of_x_aux = p * h_of_x_aux
    new_p = p_MUL_h_of_x_aux / np.sum(p_MUL_h_of_x_aux)
    prev_p = p
    p = new_p
    if False:
      print('*'*80)
      print(f"t={t}:best_h={best_h}, a_t={a_t}")
      print(f"h_of_x={h_of_x.shape}, Sy={Sy.shape}")
      print(f"h_of_x={h_of_x}, Sy={Sy}")
      print(f"p={p}")
    else:
      print(f"t={t} done")
  # end
  return H_T


In [33]:
## training
#H_T = Adaboost((Xtest, Ytest), 3)
H_T = Adaboost((Xtrain, Ytrain), 30)

t=0 done
t=1 done
t=2 done
t=3 done
t=4 done
t=5 done
t=6 done
t=7 done
t=8 done
t=9 done
t=10 done
t=11 done
t=12 done
t=13 done
t=14 done
t=15 done
t=16 done
t=17 done
t=18 done
t=19 done
t=20 done
t=21 done
t=22 done
t=23 done
t=24 done
t=25 done
t=26 done
t=27 done
t=28 done
t=29 done


In [35]:
print(H_T)

[{'idx': 406, 'theta': 1, 'dir': -1, 'err': 0.03505471956224338, 'h_of_x': array([-1,  1, -1, ...,  1,  1,  1]), 'a_t': 1.6575805694153678}, {'idx': 455, 'theta': 15, 'dir': 1, 'err': 0.11432510816336398, 'h_of_x': array([-1,  1, -1, ...,  1,  1,  1]), 'a_t': 1.0236518641911103}, {'idx': 408, 'theta': 0, 'dir': -1, 'err': 0.20261775170347643, 'h_of_x': array([-1,  1, -1, ...,  1,  1,  1]), 'a_t': 0.6850064826390513}, {'idx': 522, 'theta': 30, 'dir': 1, 'err': 0.22357657882910661, 'h_of_x': array([-1,  1, -1, ...,  1,  1,  1]), 'a_t': 0.6224720136307526}, {'idx': 461, 'theta': 0, 'dir': -1, 'err': 0.24585926069139474, 'h_of_x': array([-1,  1, -1, ...,  1,  1,  1]), 'a_t': 0.5604098731469328}, {'idx': 453, 'theta': 127, 'dir': 1, 'err': 0.23998511194360653, 'h_of_x': array([ 1,  1, -1, ...,  1,  1,  1]), 'a_t': 0.5763805673934332}, {'idx': 351, 'theta': 147, 'dir': -1, 'err': 0.26529267706939386, 'h_of_x': array([-1,  1,  1, ..., -1, -1, -1]), 'a_t': 0.5093192807526573}, {'idx': 266, 'th

In [36]:
## evaluation
def predict_with_h(h_dict, X):
  X_in_idx = X[:, h_dict['idx']]
  bool_h_of_x = (X_in_idx <= h_dict['theta'])
  h_of_x = np.ones_like(bool_h_of_x) * -h_dict['dir']
  h_of_x[bool_h_of_x] = h_dict['dir']
  #print(f"h_of_x={h_of_x.shape}")
  return h_of_x.astype(np.int8)

def predict_with_H_T(H_T, X):
  res = np.zeros((len(X),len(H_T)))
  for idx, h_dict in enumerate(H_T):
    h_of_x = predict_with_h(h_dict, X)
    res[:,idx] = h_dict['a_t']*h_of_x

  #print(f"res={res.shape}")
  summed_res = np.sum(res, axis=1)
  #print(f"summed_res={summed_res.shape}")
  final_res = np.zeros_like(summed_res).astype(np.int8)
  final_res[summed_res > 0] = 1
  final_res[summed_res < 0] = -1

  #print(summed_res.shape)
  return final_res

def prediction_err(prediction, Y):
  err = np.zeros_like(Y)
  err[prediction != Y] = 1
  #print(f"prediction={prediction.shape}")
  #print(f"err={err.shape}")
  err_rate = (float)(sum(err) / len(err))
  return err_rate

train_prediction = predict_with_H_T(H_T, Xtrain)
train_err = prediction_err(train_prediction, Ytrain)

test_prediction = predict_with_H_T(H_T, Xtest)
test_err = prediction_err(test_prediction, Ytest)

print(f"train_err={train_err}, test_err={test_err}")

train_err=0.005129958960328317, test_err=0.008700102354145343
